In [ ]:
'''
Code used for thesis submitted in partial fulfillment of the requirements for the degree of Master of Science in Data Science &
Society at the School of Humanities and Digital Science of Tilburg University.

'''

In [ ]:
#packages to be imported
import csv
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from statsmodels.compat import lzip
import statsmodels.stats.api as sms
import statistics
from sklearn.inspection import permutation_importance
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Open the CSV file & preproces data
# Source code: https://www.programiz.com/python-programming/reading-csv-files

# Function to preproces csv-file into list with lists
def preproces_data(file_name):
    file_path_begin = '/Users/annasjouknijhof/Downloads/'
    file_path_end = '.csv'
    file_path_full = file_path_begin + file_name + file_path_end

#create list with strings with all information for 1 company and 1 variable
    data = []
    with open(file_path_full, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row == []:
                continue
            else:
                row = row[0].split(';')
                #row = row[:5]
                for i in range(2,5):
                    try:
                        row[i] = float(row[i])
                    except:
                        pass
            data.append(row)
    return(data)

data = preproces_data('Import_to_Python_1')

In [ ]:
#convert data to pandas dataframe and create column names
df = pd.DataFrame(data).iloc[1:]
df.columns = ['Company name', 'Variable', '2019', '2020', '2021', 'Mean']

In [ ]:
# calculate percentages of missing values
# first check for each variable number of missing values per year

# create list with variables
def extract_list_with_variable(data):
    variable_list = []
    for i in range(1,21):
        variable_list.append(data[i][1])
    return(variable_list)

variable_list = extract_list_with_variable(data)
year = ['2019', '2020', '2021', 'Mean']

# calculate aboslute number of missing values per year and variable
def missing_values_per_year_and_company(year, variable):
    list_results = []
    for i in variable:
        tmp_list = [i]
        for j in year:
            begin_index = variable.index(i)
            indices = list(range(begin_index, 221400, 20)) 
            boolean_with_missing_value = df.iloc[indices][df.iloc[indices][j] == 'Missing value'] # selecteer rows with 'missing value'
            missing_value_count = len(boolean_with_missing_value)
            tmp_list.append(missing_value_count)
        list_results.append(tmp_list)
    return(pd.DataFrame(list_results, columns= ['Variable', '2019', '2020', '2021', 'Mean']))

absolut_number = missing_values_per_year_and_company(year, variable_list)    

# calculate percentages of missing values per year and variable
def missing_values_per_year_and_company_percentage(year, variable):
    list_results = []
    for i in variable:
        tmp_list = [i]
        for j in year:
            begin_index = variable.index(i)
            indices = list(range(begin_index, 221400, 20))
            boolean_with_missing_value = df.iloc[indices][df.iloc[indices][j] == 'Missing value'] # select from dataframe all booleans with missing value
            missing_value_count = len(boolean_with_missing_value)
            missing_value_percentage = missing_value_count / 11070 * 100
            tmp_list.append(missing_value_percentage)
        list_results.append(tmp_list)
    return(pd.DataFrame(list_results, columns= ['Variable', '2019', '2020', '2021', 'Mean']))

percentages = missing_values_per_year_and_company_percentage(year, variable_list)

summ = percentages.mean(axis=1)
percentages = percentages.round(1)
percentages.to_excel('missing_value_percentages.xlsx')

    

In [ ]:
# create table to see developments over time for missing values
# source code: https://stackoverflow.com/questions/60625159/using-pandas-dataframe-melt-with-seaborn-barplot
# source code: https://stackoverflow.com/questions/25068384/bbox-to-anchor-and-loc-in-matplotlib
# source code: https://stackoverflow.com/questions/25239933/how-to-add-a-title-to-each-subplot
# source code: https://datatofish.com/line-chart-python-matplotlib/
# source code: https://www.geeksforgeeks.org/matplotlib-pyplot-savefig-in-python/
# source code: https://stackoverflow.com/questions/11244514/modify-tick-label-text


# add the column Mean to the line graph. The code for the graph already worked for the years 2019, 2020 and 2021
# for this reasone the column Mean was renamed as 2022 to include it in the table. Eventually the name 2022 was renamed backwards
# to the right name 'Mean'. This is not the beautiful option, but a quick fix that works fine.
percentages['2022'] = percentages['Mean']

# Melt dataframe to combine the all values
percentages_melted = pd.melt(percentages, id_vars = ['Variable', 'Mean'], var_name = 'Year', value_name = 'Missing data percentage')
# value_vars = all columns that are not taken as id_vars.
# make line graph for each variable
lines = percentages_melted.groupby('Variable')

fig, ax = plt.subplots()
for i, j in lines:
    j.plot(x = 'Year', y = 'Missing data percentage', ax = ax, label = i) # ax = ax > add to ax, label = var. name

# rename axes and include legend
ax.set_xlabel('Years')
ax.set_ylabel('Missing data (%)')
ax.set_title('Missing data percentages per year')
ax.legend(loc=6, bbox_to_anchor=(1, 0.5))

# rename label '2022' back to 'mean'
labels = ['', '2019', '', '2020', '', '2021', '', 'mean', '']
ax.set_xticklabels(labels)

plt.savefig('linegraph_missing_data_percentages.png', bbox_inches = 'tight')
plt.show()


In [ ]:
# delete all missing values from the 'variable' column and replace it with the right variable name (which is possible since 
# the order of the variable names for each company is the same.)
# first company (Apple) is skipped, because all variable names are available for this company and this worked better with my code.
for i in range(20, len(df)):
    variable_index = variable_list[i % 20]
    df.iloc[i,1] = variable_index


# drop the features that have an missing value rate larger than 10% (Current Ratio, Quick Ratio, Net Income 12M FWD, Gross profit Margin
# and Current Assets - Total)
# code used from https://www.askpython.com/python-modules/pandas/pandas-isin and from https://stackoverflow.com/questions/14057007/remove-rows-not-isinx for the isin() function. 

list_for_deletion = ['CURRENT RATIO', 'QUICK RATIO', 'NET INCOME 12M FWD', 'GROSS PROFIT MARGIN', 'CURRENT ASSETS - TOTAL']
df = df[-df['Variable'].isin(list_for_deletion)].reset_index(drop=True)

# select only the rows needed for further analysis and multiple imputation (index 0,1 and 5)

df = df.iloc[:, [1,5]]

# correct format of wrong number. This went wrong during the EDA in Excel.
df.iloc[2,1] = 216.23333333

# check for correct number of unique variables in column 'Variables'. This number must equal 15.
#print(len(df['Variable'].unique()) == 15) #This is True


In [ ]:
# update list with variables for variables that are deleted
def update_variable_list(variable_list):
    variable_list_2 = []
    for i in variable_list:
        if i not in list_for_deletion:
            variable_list_2.append(i)
    return(variable_list_2)

variable_list_2 = update_variable_list(variable_list)

In [ ]:
# reformat data by pivotting the DataFrame, rename columns and reorder the variables, replace cells with 'Missing value' for NaN
# code used from: https://stackoverflow.com/questions/13148429/how-to-change-the-order-of-dataframe-columns 

# add company_number column to dataframe
repetitions = (len(df) // 15) + 2
index_list = [i for i in range((len(df) // 15) + 2) for _ in range(15)]
# or
index_list = []
for i in range(repetitions):
    for _ in range(15):
        index_list.append(i)

index_list = index_list[:len(df)]
df['Index'] = index_list

# pivot table
df = df.pivot(index = 'Index', columns='Variable')


# rename and reorder columns
columns_names = ['Board Size', 'COMMON EQTY % TOTAL ASSETS', 'DIVIDEND YIELD', 'EARNINGS PER SHR','EBIT & DEPRECIATION', 'EMPLOYEES', 'ESG Combined Score', 'NET SALES OR REVENUES', 'OPERATING INCOME','RETURN ON ASSETS', 'RETURN ON EQUITY - TOTAL (%)', 'RETURN ON INVESTED CAPITAL', 'TOTAL ASSETS', 'TOTAL DEBT', 'TOTAL DEBT % TOTAL CAPITAL/STD']
df.columns = columns_names
df = df[['ESG Combined Score', 'Board Size', 'COMMON EQTY % TOTAL ASSETS', 'DIVIDEND YIELD', 'EARNINGS PER SHR','EBIT & DEPRECIATION', 'EMPLOYEES', 'NET SALES OR REVENUES', 'OPERATING INCOME','RETURN ON ASSETS', 'RETURN ON EQUITY - TOTAL (%)', 'RETURN ON INVESTED CAPITAL', 'TOTAL ASSETS', 'TOTAL DEBT', 'TOTAL DEBT % TOTAL CAPITAL/STD']]

# replace strings with 'Missing value' for Nan
df = df.replace('Missing value', np.nan)

# first: convert alle value in DataFrame to floats (instead of objects). (source code: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html)
df = df.astype(float)


In [ ]:
# EDA

# start describing the variables seperately and write it to a table
describe_df = df.describe().round(1)
describe_df

In [ ]:
# create boxplot for each column (source code: https://stackoverflow.com/questions/51777217/how-to-plot-a-boxplot-for-each-column-in-a-dataframe)
# since the scaling of the boxplots are not really the same, a seperate boxplot for each variable has been created.
# created the Total debt % total capital/STD boxplot manually

for column in df:
    plt.figure()
    df.boxplot([column], fontsize=10)
    plot.title('Boxplot from ' + column)
    #name_to_save_boxplot = 'Boxplot ' + column + '.png'
    #plt.savefig(name_to_save_boxplot, bbox_inches = 'tight')   

# summarize all boxplots together for thesis report in one image and save them as .PNG-file
# source code: https://stackoverflow.com/questions/62404884/boxplot-with-different-y-axes-and-different-y-scales-in-seaborn
# source code: https://www.educative.io/answers/what-is-a-subplots-in-matplotlib
# source code: https://stackoverflow.com/questions/9012487/matplotlib-pyplot-savefig-outputs-blank-image
# create subplots with 5 boxplots in each row
'''
column_names = df.columns.tolist()
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(20,25))
for i, col in enumerate(column_names):
    ax = sns.boxplot(y=df[col], ax=axes.flatten()[i]) # y = import data, ax = add to subplot, see source code 1 above. [i] number of subplot.
    ax.set_ylim(df[col].min(), df[col].max()) # set limits to Y-axis
    ax.set_ylabel(col)
#name_to_save_boxplot = 'boxplot_summary.png'
#plt.savefig(name_to_save_boxplot, bbox_inches = 'tight') 
plt.show() 

'''

In [ ]:
# outlier analysis - extreme values

# first, delete all (nine) rows without any values
# provide a list of rows in the dataframe with only missing values (source code: https://datatofish.com/rows-with-nan-pandas-dataframe/)
# rows_without_values = df[df.isnull().all(axis=1)] > this line is not necessary anymore
index_rows_without_values = df[df.isnull().all(axis=1)].index
deletion_list = []
for i in index_rows_without_values:
    deletion_list.append(i)
# print(len(deletion_list)) > there are 9 companies without any values. Since these are only 9 companies, these companies are deleted from the data.
df = df.drop(deletion_list)

# delete illegal values and extreme values using the boxplots above and the describe_df DataFrame
# e.g. negative values that cannot be negative of extreme and impossible values
# values for percentages and ratio's are sooner considered outlier than variables with absolute values
# source code used from: https://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-where-column-matches-certain-value

outliers_list = []
# check for negative values
'''for i in df.columns.tolist():
    print(i)
    df_t = df[df[i]<0]
    df_t = df_t.sort_values(by=i, ascending=False)
    print(len(df_t))
    print(df_t)
    print()'''

#Net Sales/Revenues there are net sales/revenues value below zero, which is not possible

# 'ESG Combined Score'
# no illegal values

# 'Board Size'
# there are some extreme values that are possible outliers, sinces boards with more than 40 members are unrealistic
df_t = df[df['Board Size']>40]
df_t = df_t.sort_values(by='Board Size')
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)

# 'COMMON EQTY % TOTAL ASSETS'
# the first four are extreme outliers and hence unlikely
#df_t = df[df['COMMON EQTY % TOTAL ASSETS']<-7500]
#df_t = df_t.sort_values(by='COMMON EQTY % TOTAL ASSETS')
#df_t = df_t.index.values.tolist()
#outliers_list.append(df_t)

#df_t = df[df['COMMON EQTY % TOTAL ASSETS']<0]
#df_t = df_t.sort_values(by='COMMON EQTY % TOTAL ASSETS')
#df_t = df_t.iloc[0:4,:]
#df_t = df_t.index.values.tolist()
#outliers_list.append(df_t)

# 'DIVIDEND YIELD'
# There is one extreme and unlikely outlier that might be deleted based on the boxplot

df_t = df[df['DIVIDEND YIELD']>200]
df_t = df_t.sort_values(by='DIVIDEND YIELD')
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)


# 'EARNINGS PER SHR'
# delete most unlikely values for values above 30k
#df_t = df[df['EARNINGS PER SHR']>30000]
#df_t = df_t.sort_values(by='EARNINGS PER SHR')
#df_t = df_t.index.values.tolist()
#outliers_list.append(df_t)

# 'EBIT & DEPRECIATION'
# There are extreme values but no illegal values or unlikely values

# 'EMPLOYEES'
# There are extreme values but no illegal values or unlikely vlaues

# 'NET SALES OR REVENUES'
# Net sales cannot be below zero, consequantially all values below zero are considered illegal outlier


df_t = df[df['NET SALES OR REVENUES']<0]
df_t = df_t.sort_values(by='NET SALES OR REVENUES')
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)


# 'OPERATING INCOME'
# There are extreme values but no illegal values or unlikely values

# 'RETURN ON ASSETS'
# two values with really low and hence unlikely return on assets with RoA lower than 650
#df_t = df[df['RETURN ON ASSETS']<-650]
#df_t = df_t.index.values.tolist()
#outliers_list.append(df_t)

'''
# 'RETURN ON EQUITY - TOTAL (%)'
# five values with really low and hence unlikely return on equity higher than 5000 or below 10000
df_t = df[df['RETURN ON EQUITY - TOTAL (%)']<-10000]
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)
df_t = df[df['RETURN ON EQUITY - TOTAL (%)']>5000]
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)

# 'RETURN ON INVESTED CAPITAL' 
# four values with really low and hence unlikely return on invested capital below 2000
df_t = df[df['RETURN ON INVESTED CAPITAL']<-2000]
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)

# 'TOTAL ASSETS'
# There are extreme values but no illegal values or unlikely values

# 'TOTAL DEBT'
# There are extreme values but no illegal values or unlikely values

# 'TOTAL DEBT % TOTAL CAPITAL/STD'
df_t = df[df['TOTAL DEBT % TOTAL CAPITAL/STD']<-4000]
df_t = df_t.index.values.tolist()
outliers_list.append(df_t)
'''
# delete all selected outliers

outliers_list_dev = []
for i in outliers_list:
    for j in i:
        if j not in outliers_list_dev:
            outliers_list_dev.append(j)

df = df.drop(outliers_list_dev)



In [ ]:
# perform outlier analysis and scaling analysis based on splitting the data where models are trained on training data instead of
# full data
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']

# split in training and other
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)
train_data_complete = pd.concat([X_train, y_train], axis=1)
test_data_complete = pd.concat([X_test, y_test], axis=1)

In [ ]:
# Quantile Transformation > tranform data distribution of all variables except the ESG Combined Score to normal distributed value
# source code: https://towardsdatascience.com/5-data-transformers-to-know-from-scikit-learn-612bc48b8c89

# perform on training data
for i in train_data_complete.columns[1:15]:
    quantile_transformer = QuantileTransformer(random_state=0,  output_distribution='normal')
    j = i + '_trans'
    train_data_complete[j] = pd.Series(quantile_transformer.fit_transform(np.array(train_data_complete[i]).reshape(-1, 1))[:,0])
train_data_complete = train_data_complete.drop(train_data_complete.columns[1:16], axis=1)

# perform on test data
for i in test_data_complete.columns[1:15]:
    quantile_transformer = QuantileTransformer(random_state=0,  output_distribution='normal')
    j = i + '_trans'
    test_data_complete[j] = pd.Series(quantile_transformer.fit_transform(np.array(test_data_complete[i]).reshape(-1, 1))[:,0]) # see source code
test_data_complete = test_data_complete.drop(test_data_complete.columns[1:16], axis=1)

In [ ]:
# standardize all variables

from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
df_t4 = test_data_complete

# standardization of dependent variables
scaled_data = scale.fit_transform(df_t4) 
test_data_complete = pd.DataFrame(scaled_data, columns=test_data_complete.columns.tolist())

from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
df_t4 = train_data_complete

# standardization of dependent variables
scaled_data = scale.fit_transform(df_t4) 
train_data_complete = pd.DataFrame(scaled_data, columns=train_data_complete.columns.tolist())

In [ ]:
# standardize all variables except dependent variable

from sklearn.preprocessing import StandardScaler

for df in [test_data_complete, train_data_complete]:
    scale= StandardScaler()
    df_t4 = df.loc[:, df.columns != 'ESG Combined Score']
    # standardization of dependent variables
    scaled_data = scale.fit_transform(df_t4) 
    scaled_data = pd.DataFrame(scaled_data, columns=df_t4.columns.tolist())

    df = pd.concat([df.loc[:, df.columns == 'ESG Combined Score'], scaled_data], axis= 1)



In [ ]:
# semmatric logarithmicly scale all variable but the target variable (ESG Combined Score)
# for negative values it take the logarithm of the absolute value + a small value of 0.00001 multiplied by the sign of the number
# value of 0.00001 is added to make sure that none of the values equals zero.
# source code: https://stackoverflow.com/questions/29763620/how-to-select-all-columns-except-one-in-pandas
# source code: https://pandas.pydata.org/docs/user_guide/merging.html
for df in [test_data_complete, train_data_complete]:
    df_t = df.loc[:, df.columns != 'ESG Combined Score']
    df_t = np.sign(df_t) * np.log10(np.abs(df_t) + 0.00001)
    df = pd.concat([df.loc[:, df.columns == 'ESG Combined Score'], df_t], axis= 1)

In [ ]:
# semmatric logarithmicly scale all variable 
# for negative values it take the logarithm of the absolute value + a small value of 0.00001 multiplied by the sign of the number
# value of 0.00001 is added to make sure that none of the values equals zero.
# source code: https://stackoverflow.com/questions/29763620/how-to-select-all-columns-except-one-in-pandas
# source code: https://pandas.pydata.org/docs/user_guide/merging.html
for df in [test_data_complete, train_data_complete]:
    df = np.sign(df) * np.log10(np.abs(df) + 0.00001)

In [ ]:
# test for overfitting
imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete)
train_data_complete = pd.DataFrame(imputed, columns=train_data_complete.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(test_data_complete)
test_data_complete = pd.DataFrame(imputed, columns=test_data_complete.columns)

X_train = train_data_complete.drop('ESG Combined Score', axis=1)
y_train = train_data_complete['ESG Combined Score']

X_test = test_data_complete.drop('ESG Combined Score', axis=1)
y_test = test_data_complete['ESG Combined Score']

# train model again
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('R-squared: ' + str(r2_score(y_test, y_pred)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)
y_pred_train = model.predict(X_train)
MAE = metrics.mean_absolute_error(y_train, y_pred_train)
MSE = metrics.mean_squared_error(y_train, y_pred_train)
RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
print('R-squared: ' + str(r2_score(y_train, y_pred_train)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)


In [ ]:
# split data in train, validation and test set
# this is done by using the train_test function from the sklearn package twice
# source code: https://towardsdatascience.com/how-to-split-data-into-three-sets-train-validation-and-test-and-why-e50d22d3e54c
# source code: https://stackoverflow.com/questions/56166130/setting-seed-on-train-test-split-sklearn-python
# we split the data in 80:20

# select dependent and independent variables
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']

# split in training and other
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

# split x_rest en y_rest in relationship of 50:50
#X_valid, X_test, y_valid, y_test = train_test_split(X_rest,y_rest, test_size=0.5, random_state=454)

In [ ]:
# merge X en y trainingsdata sets
train_data_complete = pd.concat([X_train, y_train], axis=1)

In [ ]:
# feature selection 
#source code: https://medium.com/@szabo.bibor/how-to-create-a-seaborn-correlation-heatmap-in-python-834c0686b88e
#train_data_complete = train_data_complete.drop(['RETURN ON ASSETS'], axis = 1)
#train_data_complete = train_data_complete.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
#train_data_complete = train_data_complete.drop(['TOTAL ASSETS'], axis = 1)
#train_data_complete = train_data_complete.drop(['EBIT & DEPRECIATION'], axis = 1)
#train_data_complete = train_data_complete.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1)
#train_data_complete = train_data_complete.drop(['TOTAL DEBT'], axis = 1)
# heatmap for all combinations
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(train_data_complete.corr(),annot=True)
heatmap.set_title("Feature selection based on Pearson's correlation coefficients")
#plt.savefig('heatmap_1.png', bbox_inches = 'tight')
'''
# heatmap after dropping return on assets
df = df.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
df = df.drop(['TOTAL ASSETS'], axis = 1)
df = df.drop(['EBIT & DEPRECIATION'], axis = 1) # > dropped, because high correlation with two variables
df = df.drop(['TOTAL DEBT'], axis = 1) 
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(df.corr(),annot=True)
heatmap.set_title("Feature selection based on Pearson's correlation coefficients")
#plt.savefig('heatmap_1.png', bbox_inches = 'tight')'''


In [ ]:
X_train = train_data_complete.drop('ESG Combined Score', axis=1)
y_train = train_data_complete['ESG Combined Score']

In [ ]:
# CODE HEREAFTING SEES ON EXECUTING MODELS

R2_LR = []
MAE_LR = []
RMSE_LR = []

R2_SVR = []
MAE_SVR = []
RMSE_SVR = []

R2_RF = []
MAE_RF = []
RMSE_RF = []

overfitting_R2_LR = []
overfitting_MAE_LR = []
overfitting_RMSE_LR = []

overfitting_R2_SVR = []
overfitting_MAE_SVR = []
overfitting_RMSE_SVR = []

overfitting_R2_RF = []
overfitting_MAE_RF = []
overfitting_RMSE_RF = []

models = [777, 257, 8, 511, 2684]

for i in models:

    # select dependent and independent variables
    X = df.drop(columns = ['ESG Combined Score'])
    y = df['ESG Combined Score']

    # split in training and other
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=i)

    # random_state argument setted for reproducibility
    # Code used from: https://towardsdatascience.com/iterative-imputation-with-scikit-learn-8f3eb22b1a38 

    # drop variables
    X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
    X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
    X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
    X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

    X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
    X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
    X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
    X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_test = X_test.drop(['TOTAL DEBT'], axis = 1)

    # FROM HERE: TRAIN MODELS

    # Linear_regression

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)

    MAE_LR.append(MAE)
    RMSE_LR.append(RMSE)
    R2_LR.append(R2)

        # check for overfitting
    y_pred_train = model.predict(X_train)
    MAE = metrics.mean_absolute_error(y_train, y_pred_train)
    MSE = metrics.mean_squared_error(y_train, y_pred_train)
    RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
    R2 = r2_score(y_train, y_pred_train)
    
    overfitting_MAE_LR.append(MAE)
    overfitting_RMSE_LR.append(RMSE)
    overfitting_R2_LR.append(R2)

    # SVR
    model = SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)

    MAE_SVR.append(MAE)
    RMSE_SVR.append(RMSE)
    R2_SVR.append(R2)

        # check for overfitting
    y_pred_train = model.predict(X_train)
    MAE = metrics.mean_absolute_error(y_train, y_pred_train)
    MSE = metrics.mean_squared_error(y_train, y_pred_train)
    RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
    R2 = r2_score(y_train, y_pred_train)

    overfitting_MAE_SVR.append(MAE)
    overfitting_RMSE_SVR.append(RMSE)
    overfitting_R2_SVR.append(R2)

    # Random Forest
    model = RandomForestRegressor(n_estimators=1000, min_samples_split=2, min_samples_leaf=1, max_features=2, max_depth=15)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)

    MAE_RF.append(MAE)
    RMSE_RF.append(RMSE)
    R2_RF.append(R2)

        # check for overfitting
    y_pred_train = model.predict(X_train)
    MAE = metrics.mean_absolute_error(y_train, y_pred_train)
    MSE = metrics.mean_squared_error(y_train, y_pred_train)
    RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
    R2 = r2_score(y_train, y_pred_train)

    overfitting_MAE_RF.append(MAE)
    overfitting_RMSE_RF.append(RMSE)
    overfitting_R2_RF.append(R2)

print(R2_LR)
print(MAE_LR)
print(RMSE_LR)

print(R2_SVR)
print(MAE_SVR )
print(RMSE_SVR )

print(R2_RF )
print(MAE_RF )
print(RMSE_RF )

print(overfitting_R2_LR )
print(overfitting_MAE_LR )
print(overfitting_RMSE_LR )

print(overfitting_R2_SVR )
print(overfitting_MAE_SVR )
print(overfitting_RMSE_SVR)

print(overfitting_R2_RF)
print(overfitting_MAE_RF)
print(overfitting_RMSE_RF)

In [ ]:
import statistics

print("MLR")
print()
print("Std R2_LR:", statistics.stdev(R2_LR))
print("Mean R2_LR:", statistics.mean(R2_LR))
print()

print("Std MAE_LR:", statistics.stdev(MAE_LR))
print("Mean MAE_LR:", statistics.mean(MAE_LR))
print()

print("Std RMSE_LR:", statistics.stdev(RMSE_LR))
print("Mean RMSE_LR:", statistics.mean(RMSE_LR))
print()

print("Std overfitting_R2_LR:", statistics.stdev(overfitting_R2_LR))
print("Mean overfitting_R2_LR:", statistics.mean(overfitting_R2_LR))
print()

print("Std overfitting_MAE_LR:", statistics.stdev(overfitting_MAE_LR))
print("Mean overfitting_MAE_LR:", statistics.mean(overfitting_MAE_LR))
print()

print("Std overfitting_RMSE_LR:", statistics.stdev(overfitting_RMSE_LR))
print("Mean overfitting_RMSE_LR:", statistics.mean(overfitting_RMSE_LR))
print()

print("SVR")
print()
print("Std R2_SVR:", statistics.stdev(R2_SVR))
print("Mean R2_SVR:", statistics.mean(R2_SVR))
print()

print("Std MAE_SVR:", statistics.stdev(MAE_SVR))
print("Mean MAE_SVR:", statistics.mean(MAE_SVR))
print()

print("Std RMSE_SVR:", statistics.stdev(RMSE_SVR))
print("Mean RMSE_SVR:", statistics.mean(RMSE_SVR))
print()

print("Std overfitting_R2_SVR:", statistics.stdev(overfitting_R2_SVR))
print("Mean overfitting_R2_SVR:", statistics.mean(overfitting_R2_SVR))
print()

print("Std overfitting_MAE_SVR:", statistics.stdev(overfitting_MAE_SVR))
print("Mean overfitting_MAE_SVR:", statistics.mean(overfitting_MAE_SVR))
print()

print("Std overfitting_RMSE_SVR:", statistics.stdev(overfitting_RMSE_SVR))
print("Mean overfitting_RMSE_SVR:", statistics.mean(overfitting_RMSE_SVR))
print()

print("RF")
print()
print("Std R2_RF:", statistics.stdev(R2_RF))
print("Mean R2_RF:", statistics.mean(R2_RF))
print()

print("Std MAE_RF:", statistics.stdev(MAE_RF))
print("Mean MAE_RF:", statistics.mean(MAE_RF))
print()

print("Std RMSE_RF:", statistics.stdev(RMSE_RF))
print("Mean RMSE_RF:", statistics.mean(RMSE_RF))
print()

print("Std overfitting_R2_RF:", statistics.stdev(overfitting_R2_RF))
print("Mean overfitting_R2_RF:", statistics.mean(overfitting_R2_RF))
print()

print("Std overfitting_MAE_RF:", statistics.stdev(overfitting_MAE_RF))
print("Mean overfitting_MAE_RF:", statistics.mean(overfitting_MAE_RF))
print()

print("Std overfitting_RMSE_RF:", statistics.stdev(overfitting_RMSE_RF))
print("Mean overfitting_RMSE_RF:", statistics.mean(overfitting_RMSE_RF))




In [ ]:
X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
X_test = X_test.drop(['TOTAL DEBT'], axis = 1)

In [ ]:
# random_state argument setted for reproducibility
# Code used from: https://towardsdatascience.com/iterative-imputation-with-scikit-learn-8f3eb22b1a38 

train_data_complete_train = pd.concat([X_train, y_train], axis=1)
train_data_complete_test = pd.concat([X_test, y_test], axis=1)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_train)
train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_test)
train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


In [ ]:
# split data back

X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
y_train = train_data_complete_train['ESG Combined Score']

X_test = train_data_complete_test.drop('ESG Combined Score', axis=1)
y_test = train_data_complete_test['ESG Combined Score']

In [ ]:
# source code: https://medium.com/machine-learning-with-python/multiple-linear-regression-implementation-in-python-2de9b303fc0c
# source code for time: https://github.com/idr4n/naivebayes-email-author

model = LinearRegression()
model.fit(X_train, y_train)
print("Intercept: ", model.intercept_)
print("Coefficients:",pd.DataFrame(list(zip(X, model.coef_)))) # see source code

#Prediction of test set
y_pred = model.predict(X_test)

In [ ]:
# source code: https://medium.com/machine-learning-with-python/multiple-linear-regression-implementation-in-python-2de9b303fc0c
# source code: https://www.statology.org/r-squared-in-python/

MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('R-squared: ' + str(r2_score(y_test, y_pred)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)

In [ ]:
# test for overfitting


y_pred_train = model.predict(X_train)
MAE = metrics.mean_absolute_error(y_train, y_pred_train)
MSE = metrics.mean_squared_error(y_train, y_pred_train)
RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
print('R-squared: ' + str(r2_score(y_train, y_pred_train)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)


In [ ]:
# support vector regression - standard default settings model
# source code: https://www.section.io/engineering-education/support-vector-regression-in-python/

model = SVR()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
MAE = metrics.mean_absolute_error(y_test, y_pred)
MSE = metrics.mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print('R-squared: ' + str(r2_score(y_test, y_pred)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)

print('Check for overfitting: ')
print()
y_pred_train = model.predict(X_train)
MAE = metrics.mean_absolute_error(y_train, y_pred_train)
MSE = metrics.mean_squared_error(y_train, y_pred_train)
RMSE = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
print('R-squared: ' + str(r2_score(y_train, y_pred_train)))
print('MAE:', MAE)
print('MSE:', MSE)
print('RMSE :', RMSE)

In [ ]:
# source code: https://www.projectpro.io/recipes/find-optimal-parameters-using-gridsearchcv-for-regression
# source code: https://www.kaggle.com/code/aniketyadav1/svm-and-hyper-parameter-tuning
# source code: https://www.vebuso.com/2020/03/svm-hyperparameter-tuning-using-gridsearchcv/
# source code: https://medium.com/it-paragon/support-vector-machine-regression-cf65348b6345
# plus documentation to learn the parameters

# define hyperparameters to tune
parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}

# create a GridSearchCV object
grid_search = GridSearchCV(model, parameters, scoring='r2', n_jobs=-1)

# fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# print the best hyperparameters and the corresponding score
print(grid_search.best_params_)
print(grid_search.best_score_)
# adding positive=True results in a lower R-squared

# still not finish loading after 8555m and 37.5s
# than: tried manually
# than tried again without the kernel 'poly'
# than check if poly is better with other optimal parameters as constants and calculate for default settings

In [ ]:
# Manual hyperparameter tuning
C = [0.1, 1, 10, 100]
gamma = [1,0.1,0.01,0.001]
kernel = ['rbf', 'sigmoid', 'linear', 'poly']
epsilon = [0.1, 0.3, 0.5, 0.7, 0.9]
dataframe_input_list = []
for c in C:
    for g in gamma:
        for k in kernel:
            tmp_result_list = []
            tmp_result_list.append(c)
            tmp_result_list.append(g)
            tmp_result_list.append(k)
            model = SVR(C=c, gamma=g, kernel=k )
            model.fit(X_train, y_train)
            y_pred_test = model.predict(X_test)
            y_pred_training = model.predict(X_train)
            tmp_result_list.append(r2_score(y_train, y_pred_training))
            tmp_result_list.append(r2_score(y_test, y_pred_test))
            tmp_result_list.append(r2_score(y_test, y_pred_test) - r2_score(y_train, y_pred_training))
            dataframe_input_list.append(tmp_result_list)
            print(dataframe_input_list)
            print(tmp_result_list)

dataframe_output = pd.DataFrame(dataframe_input_list)
dataframe_output

# Conclusion: the Poly-kernel is problematic due to loading times.

In [ ]:
# Try GridSearchCV again, but without Poly-kernel and with more hyperparameters, as the model now works fast
parameters = {'C': [0.0625, 0.5, 1, 2, 16, 128], 'gamma': [0.001953, 0.03125, 0.125, 0.5, 1], 'kernel': ['rbf', 'sigmoid', 'linear'], 'epsilon': [0.1, 0.25, 0.5, 1]}
grid_search = GridSearchCV(model, parameters, scoring='r2', n_jobs=-1) 
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
# Create the parameter grid based on the results of random search  >  final
param_grid = {
    'max_depth': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    'max_features': [1, 2, 3, 4, 5, 6],
    'min_samples_leaf': [1,5, 25, 50, 100, 200, 300, 400, 500, 600],
    'min_samples_split': [1, 2, 3, 4, 5, 6],
    'n_estimators': [1,5,10,50,100, 200, 300, 400, 500, 750, 1000]
}
# Create a based model
model = RandomForestRegressor()
# Instantiate the grid search model
random_search = RandomizedSearchCV(estimator = model, scoring='r2', param_distributions= param_grid, n_iter=10000, n_jobs = -1)
random_search.fit(X_train, y_train)
print(random_search.best_params_)
print(random_search.best_score_)


# {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 2, 'max_depth': 15}
# 0.3531047891460169


In [ ]:
# Result van een gridsearchCV met de onderstaande parameters
''' 
op trainingsdata zonder validatiedata
'max_depth': [5, 10, 25, 50],
'max_features': [2,4,6],
'min_samples_leaf': [1,5,10],
'min_samples_split': [2,4,6],
'n_estimators': [1,5,10,50,100]
{'max_depth': 25, 'max_features': 2, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 100}'''
# took 83m 39s

# 1530 minutes training time > stopped with following parameters
'''   
'max_depth': [5, 10, 25, 50],
'max_features': [2,4,6],
'min_samples_leaf': [1,2,5,10,50,100,300],
'min_samples_split': [2,4,6],
'n_estimators': [1,5,10,50,100,300]'''


'''
Resulted in:
'max_depth': [5, 15, 25, 35, 50],
'max_features': [2,4,6],
'min_samples_leaf': [1,5, 25, 100, 600],
'min_samples_split': [2,4,6],
'n_estimators': [1,5,10,50,100]
    
Results for above hyperparameters on training data (no validation data present)
{'max_depth': 15,
 'max_features': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}
 >>> minimum sample leaf tunen geeft een veel beter resultaat met minder overfitting'''

In [ ]:
# create subplot with three plots including all residual plots for each model
# source code: https://www.geeksforgeeks.org/plot-multiple-plots-in-matplotlib/ & documentation of subplots function
# source code for residual plot: https://datagy.io/seaborn-residplot/
# used for averaging scores: https://www.javatpoint.com/how-to-add-two-lists-in-python 

models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=100, min_samples_leaf=1, max_features = 6, min_samples_split = 2, random_state=19)]
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
x = 0
for model in models:
    '''prediction_lists = []

    for i in [8, 777, 257, 511, 2684]:
        # select dependent and independent variables
        X = df.drop(columns = ['ESG Combined Score'])
        y = df['ESG Combined Score']

        # split in training and other
        X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=i)

        # random_state argument setted for reproducibility
        # Code used from: https://towardsdatascience.com/iterative-imputation-with-scikit-learn-8f3eb22b1a38 

        # drop variables
        X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
        X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
        X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
        X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
        X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
        X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

        X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
        X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
        X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) # HAS 2 IPV 1 CORRELATING VARIABLES
        X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
        X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
        X_test = X_test.drop(['TOTAL DEBT'], axis = 1)

        train_data_complete_train = pd.concat([X_train, y_train], axis=1)
        train_data_complete_test = pd.concat([X_test, y_test], axis=1)

        imputer = IterativeImputer(random_state=42)
        imputed = imputer.fit_transform(train_data_complete_train)
        train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

        imputer = IterativeImputer(random_state=42)
        imputed = imputer.fit_transform(train_data_complete_test)
        train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)

        X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
        y_train = train_data_complete_train['ESG Combined Score']

        X_test = train_data_complete_test.drop('ESG Combined Score', axis=1)
        y_test = train_data_complete_test['ESG Combined Score']

        fit = model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        prediction_lists.append(list(y_pred))
    lists_total = []
    for i in range(len(prediction_lists[0])):
        total = sum(prediction_lists[j][i] for j in range(len(prediction_lists)))
        lists_total.append(total / 5)
    y_pred = [value / len(prediction_lists) for value in y_pred]
    # after this y_pred is averaged for the model'''

    residuals = y_test - y_pred
    sns.regplot(data=residuals, x=y_pred, y=residuals, ax=ax[x])
    ax[x].axhline(y=0, color='red')
    ax[x].set_xlabel('Predicted value')
    ax[x].set_ylabel('Residual')
    if x == 0:
        ax[x].set_title('Residual plot for MLR model')
    elif x == 1:
        ax[x].set_title('Residual plot for SVR model')
    else:
        ax[x].set_title('Residual plot for RF')
    x += 1
plt.savefig('residual_plots_all_models', bbox_inches = 'tight') 
plt.show()

In [ ]:
# create subplot with three plots including all residual plots for each model
# source code: https://www.geeksforgeeks.org/plot-multiple-plots-in-matplotlib/ & documentation of subplots function
# source code for residual plot: https://datagy.io/seaborn-residplot/
# used for averaging scores: https://www.javatpoint.com/how-to-add-two-lists-in-python 

models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=1000, min_samples_leaf=1, max_features = 2, min_samples_split = 2, random_state=19)]
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
x = 0
for model in models:
    fit = model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    sns.regplot(data=residuals, x=y_pred, y=residuals, ax=ax[x])
    ax[x].axhline(y=0, color='red')
    ax[x].set_xlabel('Predicted value')
    ax[x].set_ylabel('Residual')
    if x == 0:
        ax[x].set_title('Residual plot for MLR model')
    elif x == 1:
        ax[x].set_title('Residual plot for SVR model')
    else:
        ax[x].set_title('Residual plot for RF')
    x += 1
plt.savefig('residual_plots_all_models', bbox_inches = 'tight') 
plt.show()

In [ ]:
# execute Breusch-Pagan test to test statistically for the presence of heteroscedasticity 
# source code: https://www.statology.org/breusch-pagan-test-python/
# source code: https://www.geeksforgeeks.org/how-to-perform-a-breusch-pagan-test-in-python/

models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=1000, min_samples_leaf=1, max_features = 2, min_samples_split = 2, random_state=19)] 
for i, model in enumerate(models):
    if i == 0:
        print('MLR model')
    elif i == 1:
        print('SVR model')
    else:
        print('RF')
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    residuals = y_test - y_pred
    names = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
    test_result = sms.het_breuschpagan(residuals, X_test) # X_test, because suspected for heteroscedasticity
    print(lzip(names, test_result)) # creates tupple with name and test_result

In [ ]:
# error analysis > 4 groups based on ESG rating
# test for difference in companies with lower and higher ESG-ratings
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
X_test = X_test.drop(['TOTAL DEBT'], axis = 1)

train_data_complete_train = pd.concat([X_train, y_train], axis=1)
train_data_complete_test = pd.concat([X_test, y_test], axis=1)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_train)
train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_test)
train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
y_train = train_data_complete_train['ESG Combined Score']

X_test = train_data_complete_test.drop('ESG Combined Score', axis=1)
y_test = train_data_complete_test['ESG Combined Score']
# first create two lists with all the y and X rows.
# check performance on four classes of ESG scores (lowest 25%, 25 till 50% 50 till 75% and 75% percent)
# source code to select the right columns: https://stackoverflow.com/questions/34243194/filter-rows-of-pandas-dataframe-whose-values-are-lower-than-0
# select break points 
Q1 = y_test.quantile(0.25)
median = y_test.median()
Q3 = y_test.quantile(0.75)

# select groups in y_test set based on break points
y_under_25 = y_test[y_test <= Q1]
y_between_25_and_50 = y_test[(y_test > Q1) & (y_test <= median)]
y_between_50_and_75 = y_test[(y_test > median) & (y_test <= Q3)]
y_over_75 = y_test[y_test > Q3]
X_under_25 = None
X_between_25_and_50 = None
X_between_50_and_75 = None
X_over_75 = None
X_groups = [X_under_25, X_between_25_and_50, X_between_50_and_75, X_over_75]
# select right independent values per group 
y_groups = [y_under_25, y_between_25_and_50, y_between_50_and_75, y_over_75]

for i, group in enumerate(y_groups):
    group_indices = group.index.tolist()
    X_groups[i] = X_test[X_test.index.isin(group_indices)]

In [ ]:
# train models on training data set
models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=1000, min_samples_leaf=1, max_features = 2, min_samples_split = 2, random_state=19)] 
for model in models:
    X = df.drop(columns = ['ESG Combined Score'])
    y = df['ESG Combined Score']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

    X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
    X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
    X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

    X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
    X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
    X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
    train_data_complete_train = pd.concat([X_train, y_train], axis=1)
    train_data_complete_test = pd.concat([X_test, y_test], axis=1)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_train)
    train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_test)
    train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


    X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
    y_train = train_data_complete_train['ESG Combined Score']
    model.fit(X_train, y_train)
    for i in range(0,4):
        X_test = X_groups[i]
        y_test = y_groups[i]
        y_pred = model.predict(X_test)
        print(model)
        print('quantile ' + str(i))
        print()
        MAE = metrics.mean_absolute_error(y_test, y_pred)
        RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        print('R-squared: ' + str(r2_score(y_test, y_pred)))
        print('MAE:', MAE)
        print('RMSE :', RMSE)
        print()

In [ ]:
# error analysis > based on ESG with only two groups: higher and lower 
# test for difference in companies with lower and higher ESG-ratings
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
train_data_complete_train = pd.concat([X_train, y_train], axis=1)
train_data_complete_test = pd.concat([X_test, y_test], axis=1)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_train)
train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_test)
train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
y_train = train_data_complete_train['ESG Combined Score']
# first create two lists with all the y and X rows.
# check performance on four classes of ESG scores (lowest 25%, 25 till 50% 50 till 75% and 75% percent)
# source code to select the right columns: https://stackoverflow.com/questions/34243194/filter-rows-of-pandas-dataframe-whose-values-are-lower-than-0
# select break point
median = y_test.median()

# select groups in y_test set based on break points
y_lower = y_test[y_test < median]
y_higher = y_test[y_test >= median]
X_lower = None
X_higher = None
X_groups = [X_lower, X_higher]
# select right independent values per group 
y_groups = [y_lower, y_higher]

for i, group in enumerate(y_groups):
    group_indices = group.index.tolist()
    X_groups[i] = X_test[X_test.index.isin(group_indices)]


In [ ]:
models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=100, min_samples_leaf=1, max_features = 6, min_samples_split = 2, random_state=19)] 
for model in models:
    X = df.drop(columns = ['ESG Combined Score'])
    y = df['ESG Combined Score']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

    X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
    X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
    X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

    X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
    X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
    X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
    train_data_complete_train = pd.concat([X_train, y_train], axis=1)
    train_data_complete_test = pd.concat([X_test, y_test], axis=1)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_train)
    train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_test)
    train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


    X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
    y_train = train_data_complete_train['ESG Combined Score']
    model.fit(X_train, y_train)
    for i in range(0,2):
        X_test = X_groups[i]
        y_test = y_groups[i]
        y_pred = model.predict(X_test)
        print(model)
        print('split_number: ' + str(i))
        print()
        MAE = metrics.mean_absolute_error(y_test, y_pred)
        RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        print('R-squared: ' + str(r2_score(y_test, y_pred)))
        print('MAE:', MAE)
        print('RMSE :', RMSE)
        print()

In [ ]:
# error analysis > 4 groups based on revenue
# test for difference between larger and smaller companies based on revenues
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
X_test = X_test.drop(['TOTAL DEBT'], axis = 1)

train_data_complete_train = pd.concat([X_train, y_train], axis=1)
train_data_complete_test = pd.concat([X_test, y_test], axis=1)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_train)
train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_test)
train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
y_train = train_data_complete_train['ESG Combined Score']
# source code to select the right columns: https://stackoverflow.com/questions/34243194/filter-rows-of-pandas-dataframe-whose-values-are-lower-than-0
# select break points 
Q1 = X_test['NET SALES OR REVENUES'].quantile(0.25)
median = X_test['NET SALES OR REVENUES'].median()
Q3 = X_test['NET SALES OR REVENUES'].quantile(0.75)

# select groups in y_test set based on break points
y_under_25 = None
y_between_25_and_50 = None
y_between_50_and_75 = None
y_over_75 = None

X_under_25 = X_test[X_test['NET SALES OR REVENUES'] <= Q1]
X_between_25_and_50 = X_test[(X_test['NET SALES OR REVENUES'] > Q1) & (X_test['NET SALES OR REVENUES'] <= median)]
X_between_50_and_75 = X_test[(X_test['NET SALES OR REVENUES'] > median) & (X_test['NET SALES OR REVENUES'] <= Q3)]
X_over_75 = X_test[X_test['NET SALES OR REVENUES'] > Q3]
X_groups = [X_under_25, X_between_25_and_50, X_between_50_and_75, X_over_75]
y_groups = [y_under_25, y_between_25_and_50, y_between_50_and_75, y_over_75]

# select right independent values per group 
for i, group in enumerate(X_groups):
    group_indices = group.index.tolist()
    y_groups[i] = y_test[y_test.index.isin(group_indices)]

In [ ]:
models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=100, min_samples_leaf=1, max_features = 6, min_samples_split = 2, random_state=19)] 
for model in models:
    X = df.drop(columns = ['ESG Combined Score'])
    y = df['ESG Combined Score']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

    X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
    y_train = train_data_complete_train['ESG Combined Score']

    X_test = train_data_complete_test.drop('ESG Combined Score', axis=1)
    y_test = train_data_complete_test['ESG Combined Score']

    X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
    X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
    X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

    X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
    X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
    X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
    train_data_complete_train = pd.concat([X_train, y_train], axis=1)
    train_data_complete_test = pd.concat([X_test, y_test], axis=1)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_train)
    train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_test)
    train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


    X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
    y_train = train_data_complete_train['ESG Combined Score']
    model.fit(X_train, y_train)
    for i in range(0,4):
        X_test = X_groups[i]
        y_test = y_groups[i]
        y_pred = model.predict(X_test)
        print(model)
        print('quantile ' + str(i))
        print()
        MAE = metrics.mean_absolute_error(y_test, y_pred)
        RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        print('R-squared: ' + str(r2_score(y_test, y_pred)))
        print('MAE:', MAE)
        print('RMSE :', RMSE)
        print()

In [ ]:
# error analysis > based on ESG with only two groups: higher and lower 
# test for difference in companies with lower and higher ESG-ratings
X = df.drop(columns = ['ESG Combined Score'])
y = df['ESG Combined Score']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
train_data_complete_train = pd.concat([X_train, y_train], axis=1)
train_data_complete_test = pd.concat([X_test, y_test], axis=1)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_train)
train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

imputer = IterativeImputer(random_state=42)
imputed = imputer.fit_transform(train_data_complete_test)
train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
y_train = train_data_complete_train['ESG Combined Score']
# first create two lists with all the y and X rows.
# check performance on four classes of ESG scores (lowest 25%, 25 till 50% 50 till 75% and 75% percent)
# source code to select the right columns: https://stackoverflow.com/questions/34243194/filter-rows-of-pandas-dataframe-whose-values-are-lower-than-0
# select break point
median = X_test['NET SALES OR REVENUES'].median()
print(median)
# select groups in y_test set based on break points
y_lower = None
y_higher = None
X_lower = X_test[X_test['NET SALES OR REVENUES'] < median]
X_higher = X_test[X_test['NET SALES OR REVENUES'] >= median]
X_groups = [X_lower, X_higher]
y_groups = [y_lower, y_higher]

for i, group in enumerate(X_groups):
    group_indices = group.index.tolist()
    y_groups[i] = y_test[y_test.index.isin(group_indices)]


In [ ]:
models = [LinearRegression(), SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf'), RandomForestRegressor(max_depth = 15, n_estimators=100, min_samples_leaf=1, max_features = 6, min_samples_split = 2, random_state=19)] 
for model in models:
    X = df.drop(columns = ['ESG Combined Score'])
    y = df['ESG Combined Score']
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=777)

    X_train = X_train.drop(['RETURN ON ASSETS'], axis = 1)
    X_train = X_train.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_train = X_train.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_train = X_train.drop(['TOTAL ASSETS'], axis = 1)
    X_train = X_train.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_train = X_train.drop(['TOTAL DEBT'], axis = 1)

    X_test = X_test.drop(['RETURN ON ASSETS'], axis = 1)
    X_test = X_test.drop(['RETURN ON INVESTED CAPITAL'], axis = 1)
    X_test = X_test.drop(['RETURN ON EQUITY - TOTAL (%)'], axis = 1) 
    X_test = X_test.drop(['TOTAL ASSETS'], axis = 1)
    X_test = X_test.drop(['EBIT & DEPRECIATION'], axis = 1)
    X_test = X_test.drop(['TOTAL DEBT'], axis = 1)
    train_data_complete_train = pd.concat([X_train, y_train], axis=1)
    train_data_complete_test = pd.concat([X_test, y_test], axis=1)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_train)
    train_data_complete_train = pd.DataFrame(imputed, columns=train_data_complete_train.columns)

    imputer = IterativeImputer(random_state=42)
    imputed = imputer.fit_transform(train_data_complete_test)
    train_data_complete_test = pd.DataFrame(imputed, columns=train_data_complete_test.columns)


    X_train = train_data_complete_train.drop('ESG Combined Score', axis=1)
    y_train = train_data_complete_train['ESG Combined Score']
    model.fit(X_train, y_train)
    for i in range(0,2):
        X_test = X_groups[i]
        y_test = y_groups[i]
        y_pred = model.predict(X_test)
        print(model)
        print('split_number: ' + str(i))
        print()
        MAE = metrics.mean_absolute_error(y_test, y_pred)
        RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
        print('R-squared: ' + str(r2_score(y_test, y_pred)))
        print('MAE:', MAE)
        print('RMSE :', RMSE)
        print()

In [ ]:
# feature importance linear regression - using coefficients of the model
# source code: https://machinelearningmastery.com/calculate-feature-importance-with-python/
# source code to add labels: https://stackoverflow.com/questions/21910986/why-set-xticks-doesnt-set-the-labels-of-ticks
# source code to change axes from vertical to horizontal: https://www.geeksforgeeks.org/matplotlib-pyplot-barh-function-in-python/

from matplotlib import pyplot
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
importance = model.coef_
importance = np.sort(importance)
fig, ax = pyplot.subplots()
fig.set_size_inches(2.5, 3.5)
plt.title(label='MLR model feature importance')
ax.barh(range(len(importance)), importance) # y-axis = range, values = importance
ax.set_yticks(range(len(importance))) # tick size
ax.set_yticklabels(X_train.columns) # tick value
plt.savefig('MLR model feature importance', bbox_inches = 'tight') 
pyplot.show()


In [ ]:
# feature importance analysis - Support Vector Regression - using Permutation feature importance since no model specific 
# options are available
# source code: https://stackoverflow.com/questions/41592661/determining-the-most-contributing-features-for-svm-classifier-in-sklearn
# source code: https://stackoverflow.com/questions/70467781/feature-importance-with-svr

model = SVR(C=16, epsilon=1, gamma=0.125, kernel='rbf')
model = RandomForestRegressor(max_depth = 15, n_estimators=1000, min_samples_leaf=1, max_features = 2, min_samples_split = 2, random_state=19)
model.fit(X_train, y_train)
results = permutation_importance(model, X_train, y_train, scoring='neg_mean_squared_error', n_jobs=-1)
importance = results.importances_mean
indices = np.argsort(importance)
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 3.5)
plt.title(label='RF model feature importance')
ax.barh(range(len(importance)), importance[indices]) # range(len(importance)) provides place on y-axis, imporatance[indices] the order
ax.set_yticks(range(len(importance)))
ax.set_yticklabels(np.array(X_train.columns)[indices]) 
plt.savefig('RF model feature importance', bbox_inches = 'tight')
pyplot.show()

In [ ]:
# feature importance analysis - Random Forest - using the importance output of the model
# sourcde code: copied without changes from line 6 and further on may 11th 2023 from https://inria.github.io/scikit-learn-mooc/python_scripts/dev_features_importance.html
# source code: https://stackoverflow.com/questions/44511636/plot-feature-importance-with-feature-names
# source code to resize graph

# not useful anymore

model = RandomForestRegressor(max_depth = 15, n_estimators=100, min_samples_leaf=1, max_features = 6, min_samples_split = 2, random_state=19)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
importances = model.feature_importances_
indices = np.argsort(importances)
fig, ax = plt.subplots()
ax.barh(range(len(importances)), importances[indices])
ax.set_yticks(range(len(importances)))
fig.set_size_inches(2.5, 3.5)
plt.title(label='RF feature importance')
ax.set_yticklabels(np.array(X_train.columns)[indices])
plt.savefig('feature-importance - random forest', bbox_inches = 'tight') 
plt.show()

In [ ]:
# test for difference between training and test set
df_train = X_train.describe()
df_test = X_test.describe()
print(df_train)
print()
print('df_test')
print(df_test)
print(

)
[print('minus_table')]
print(df_train - df_test)

In [ ]:
'''ALL CODE AFTER THIS LINE IS NOT USED IN THE FINAL THESIS BUT ARE MAINLY TRY-OUTS (NOT USED IN THE THESIS)'''

In [ ]:
# Manual hyperparameter tuning
# in principle take default, unless specified otherwise
# source code: https://realpython.com/iterate-through-dictionary-python/#iterating-through-items
# source code: https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments
# optimize as discussed for R-squared

parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid', 'linear'], 'epsilon': [0.1, 0.3, 0.5, 0.7, 0.9]}
for name, value in parameters.items():
    for i in value:
        model = SVR()
        model.fit(X_train, y_train)
        y_pred_test = model.predict(X_test)
        y_pred_training = model.predict(X_train)
        print('Model based on: ' + name + ' = ' + str(i))
        print('R-squared on test set: ' + str(r2_score(y_train, y_pred_training)))
        print('R-squared on test set: ' + str(r2_score(y_test, y_pred_test)))
        print('The difference between R-squared scores is: ' + str(r2_score(y_test, y_pred_test) - r2_score(y_train, y_pred_training)))
        print()

# initial model, but overfitting model = SVR(C= 100, gamma= 0.1, kernel= 'rbf', epsilon= 0.3 ) > C=1 geeft ook prima result en c=10 ook, maar die overfit ook wel
# solution, decrease gamma to 0.01

In [ ]:

parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid', 'linear'], 'epsilon': [0.1, 0.3, 0.5, 0.7, 0.9]}
C = [0.1, 1, 10, 100]
gamma = [1,0.1,0.01,0.001]
kernel = ['rbf', 'sigmoid', 'linear']
epsilon = [0.1, 0.3, 0.5, 0.7, 0.9]
dataframe_input_list = []
for c in C:
    for g in gamma:
        for k in kernel:
            tmp_result_list = []
            tmp_result_list.append(c)
            tmp_result_list.append(g)
            tmp_result_list.append(k)
            model = SVR(C=c, gamma=g, kernel=k )
            model.fit(X_train, y_train)
            y_pred_test = model.predict(X_test)
            y_pred_training = model.predict(X_train)
            tmp_result_list.append(r2_score(y_train, y_pred_training))
            tmp_result_list.append(r2_score(y_test, y_pred_test))
            tmp_result_list.append(r2_score(y_test, y_pred_test) - r2_score(y_train, y_pred_training))
            dataframe_input_list.append(tmp_result_list)
            print(dataframe_input_list)
            print(tmp_result_list)

dataframe_output = pd.DataFrame(dataframe_input_list)
dataframe_output
            
            

In [ ]:
C = [0.1, 1, 10, 100]
gamma = [1,0.1,0.01,0.001]
kernel = ['rbf', 'sigmoid', 'linear']
epsilon = [0.1, 0.3, 0.5, 0.7, 0.9]
dataframe_input_list = []
x = 0
for c in C:
    for g in gamma:
        for k in kernel:
            x += 1
print(x)

In [ ]:
'''# This method was tried, but based on literature this method is not the best option to use so it is not used in the rest of the 
# project

# Apply multiple imputation for missing values with KNN-method
# This option doesn't work, since values must be normalized, but due to a wide range of possible values this unfortunately
# is impossible.


# data used from: https://datagy.io/pandas-normalize-column/ >>> let op: dit moet ook nog per column gaan gebeuren!!!!
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaled = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled, columns=df.columns)
scaled_df

# code used, based on Python documentation (https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html#sklearn.impute.KNNImputer)
imputer = KNNImputer(n_neighbors=10)
df_imputed_KNNImputer = imputer.fit_transform(df)
df_imputed_KNNImputer = pd.DataFrame(df_imputed_KNNImputer)
df_imputed_KNNImputer'''

In [ ]:
# create dataframe where probable outliers are deleted
def test(var_name):
    df_t_list = []
    for i in var_name:   
        #determine ranges
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        # filter outliers from dataset
        filter = (df[i] >= Q1 - 3 * IQR) & (df[i] <= Q3 + 3 *IQR)
        df_t = df.loc[filter]
        df_t_list.append(i)
        df_t_list.append(df_t)
    return df_t_list

df_t_list = test(df.columns.tolist())
for i in df_t_list: 
    print(i)

In [ ]:
# for many variable, there are relatively much outliers with legal, but extreme values. Therefor the Median aboslute deviation method has been used to get all the outliers.
# source code: https://blog.gitnux.com/code/pandas-quantile/

def define_data_without_outliers(df, variables_in_data, cut_of_point):
    
    for i in variables_in_data:
        #determine ranges
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1

        # filter outliers from dataset
        filter = (df[i] >= Q1 - cut_of_point * IQR) & (df[i] <= Q3 + cut_of_point *IQR)
        df_t = df.loc[filter]
        print('Filtered data for outliers of variable ' + i + ' with cut-of point ' + str(cut_of_point))    
        print("hoi")
        print(df_t)
    return(df_t)
    
print(define_data_without_outliers(df, ['RETURN ON ASSETS', 'ESG Combined Score'], 3))
#print(df_t.iloc[500:1500,:])

In [ ]:
# create dataframe where probable outliers are deleted
# source code: https://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-where-column-matches-certain-value 
# and https://blog.gitnux.com/code/pandas-quantile/
def create_index_list_per_var(var_name):
    df_t_list = []
    indice_list = []
    for i in var_name:   
        #determine ranges
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        # filter outliers from dataset
        filter = (df[i] >= Q1 - 3 * IQR) & (df[i] <= Q3 + 3 *IQR)
        indice = df[filter].index.tolist()
        indice_list.append(indice)
    return df_t_list, indice_list

df_t_list, indice_list = create_index_list_per_var(df.columns.tolist())

print(indice_list)

def create_list_with_outliers(indice_list):
    prob_outlier_list = []
    for i in range(0,15): 
        for j in range(0,11069): 
            if j not in indice_list:
                prob_outlier_list.append(j)
    return prob_outlier_list

outlier_indices = create_list_with_outliers(indice_list)
print(len(outlier_indices))






In [ ]:
# source code: https://note.nkmk.me/en/python-pandas-multiple-conditions/ 
var_under_research = 'Board Size'
cut_of_point = 3

Q1 = df[var_under_research].quantile(0.25)
Q3 = df[var_under_research].quantile(0.75)
IQR = Q3 - Q1
lower_fence = Q1 - cut_of_point * IQR
upper_fence = Q3 + cut_of_point * IQR

df_or = df[(df[var_under_research] < lower_fence) | (df[var_under_research] > upper_fence)]

print(len(df_or))
print(lower_fence)
print(upper_fence)
# filter outliers from dataset
df[var_under_research] >= Q1 - 3 * IQR 
df[var_under_research] <= Q3 + 3 * IQR


df_or.sort_values(by=var_under_research, ascending=False)

In [ ]:
# create DataFrame with the number of outliers with a cut-of-point of 3

# source code: https://note.nkmk.me/en/python-pandas-multiple-conditions/ 
# and https://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-where-column-matches-certain-value 
# and https://blog.gitnux.com/code/pandas-quantile/
import pandas as pd
def outliers_per_variable(var_list, cut_of_point): 
        dataframe_input = []
        for i in var_list:
            tmp_list = []
            Q1 = df[i].quantile(0.25)
            Q3 = df[i].quantile(0.75)
            IQR = Q3 - Q1
            lower_fence = Q1 - cut_of_point * IQR
            upper_fence = Q3 + cut_of_point * IQR
            df_or = df[(df[i] < lower_fence) | (df[i] > upper_fence)]
            tmp_list.append(i)
            tmp_list.append(cut_of_point)
            tmp_list.append(lower_fence)
            tmp_list.append(upper_fence)
            tmp_list.append(len(df_or))
            dataframe_input.append(tmp_list)
        outlier_statistics = pd.DataFrame(dataframe_input, columns=['variable_name', 'cut_of_point', 'lower_fence', 'upper_fence', 'number_of_outliers'])
        return(outlier_statistics)

df_outlier = outliers_per_variable(df.columns.tolist(), 3)
df_outlier

# conclusion: there are some variables with a lot of 'outliers'. However, these outliers are possibly legal outliers. 
# therefore we delete the rows that are outlier with a maximum of 100 rows per variable, since the boxplots provided the 
# the insight that for most variables there are only a view outliers between the outliers.


In [ ]:
# convenient overview to gain insights on which variables are considered outlier.
# source code: https://note.nkmk.me/en/python-pandas-multiple-conditions/ 
# and https://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-where-column-matches-certain-value 
# and https://blog.gitnux.com/code/pandas-quantile/
# and https://stackoverflow.com/questions/29077188/absolute-value-for-column-in-python

import pandas as pd
def outliers_per_variable(var_list, cut_of_point, number_outliers_deleted): 
        dataframe_input = []
        index_numbers_of_outliers = []
        for i in var_list:
            tmp_list = []
            Q1 = df[i].quantile(0.25)
            Q3 = df[i].quantile(0.75)
            IQR = Q3 - Q1
            lower_fence = Q1 - cut_of_point * IQR
            upper_fence = Q3 + cut_of_point * IQR
            df_or = df[(df[i] < lower_fence) | (df[i] > upper_fence)]
            df_or = df_or.abs().nlargest(number_outliers_deleted, i)
            df_or = df_or.sort_values(by=i, ascending=False)
            print(i)
            print('de lengte van df_or is ', str(len(df_or)))
            print(df_or)
            indices = df_or.index.tolist()
            print('indices_van de variabele ' + i + 'zijn gelijk aan ' + str(indices) + '. De lengte hiervan is ' + str(len(indices)))
        return(df_or)

df_outlier = outliers_per_variable(df.columns.tolist(), 3, 20)


In [ ]:
# convenient overview to gain insights on which variables are considered outlier.
# source code: https://note.nkmk.me/en/python-pandas-multiple-conditions/ 
# and https://stackoverflow.com/questions/21800169/python-pandas-get-index-of-rows-where-column-matches-certain-value 
# and https://blog.gitnux.com/code/pandas-quantile/
# and https://stackoverflow.com/questions/29077188/absolute-value-for-column-in-python

import pandas as pd
def outliers_per_variable(var_list, cut_of_point, number_outliers_deleted): 
        dataframe_input = []
        index_numbers_of_outliers = []
        for i in var_list:
            tmp_list = []
            Q1 = df[i].quantile(0.25)
            Q3 = df[i].quantile(0.75)
            IQR = Q3 - Q1
            lower_fence = Q1 - cut_of_point * IQR
            upper_fence = Q3 + cut_of_point * IQR
            df_or = df[(df[i] < lower_fence) | (df[i] > upper_fence)]
            df_or = df_or.abs().nlargest(number_outliers_deleted, i)
            df_or = df_or.sort_values(by=i, ascending=False)
            indices = df_or.index.tolist()
            for j in indices:
                  if j not in index_numbers_of_outliers:
                        index_numbers_of_outliers.append(j)
        return(index_numbers_of_outliers)

index_numbers_of_outliers = outliers_per_variable(df.columns.tolist(), 3, 35)

In [ ]:
# determine optimal number of maximum outliers per variable
def research_outlier_boundary_2(start, end, steps, cut_of_point_list):
    counter = 0
    for i in range(start, end,steps):
        for j in cut_of_point_list:
            target_number = len(outliers_per_variable(df.columns.tolist(),j,i))
            tmp_var = target_number - counter
            counter = target_number
            print('max number of possible instances with more than one outlier: ' + str((i*15) - target_number) + ' cut-of point: ' + str(j) + ', max number of outliers: ' + str(i) + ', total deleted instances: ' + str(target_number) + ', difference with previous number of deleted instances: ' + str(tmp_var))


print(research_outlier_boundary_2(0, 105, 5, [3]))

# conclusion: when the max number of outliers increases, eventually the difference of the number of extra deleted instances stays
# around 30. The last larger dropt is when the max number of outliers increases from 30 to 35. Therefore, 35 is chosen. 319 outliers
# isn't to many so this is considered fine. 

In [ ]:
# deal with the outliers
# rows of outliers are deleted since the max number of possible instances with more than one outlier increases if the max
# number of outliers increases. This could mean that other, not as outlier classified values of an instance with an outlier
# might involve nearly outlier values. Therefore multiple imputation is less trustworthy and the instances are deleted.

df = df.drop(index_numbers_of_outliers)

In [ ]:
'''#create list is company names
# provides a list of names of all companies included, but gives an error.
def create_list_of_companies(df):
    tmp_list = []
    for i in range(15, len(df)):
        company_name = df.iloc[i,0]
        if company_name == 'Error in name':
            continue
        elif company_name not in tmp_list:
                tmp_list.append(df.iloc[i,0])
    return(tmp_list)

company_name_list = create_list_of_companies(df)  

def replace_company_names(company_name_list):
    counter = 0
    for i in range(15, len(df), 15):
        index_number = min(counter, (len(df)/15-1))
        company_name = company_name_list[index_number]
        for j in range(0,15):
            row_number = i + j
            print(row_number)
            print(company_name)
        counter += 1


        
        

print(replace_company_names(company_name_list))'''